In [1]:
import pandas as pd
import csv

C:\Users\l3n3c\AppData\Local\Temp\ipykernel_47876\4056139722.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
#creation of SQLite database
import sqlite3

conn = sqlite3.connect('etl_project.db')
cursor = conn.cursor()

In [13]:
# creating tables
cursor.execute("CREATE TABLE IF NOT EXISTS songs (song text primary key, day integer, month integer, year integer, streams integer)")

cursor.execute("CREATE TABLE IF NOT EXISTS artists_songs (artist_name text not null, song text primary key, CONSTRAINT fk_song FOREIGN KEY (song) REFERENCES songs(song))")
cursor.execute("CREATE TABLE IF NOT EXISTS lyrics (song text primary key, lyrics text not null, CONSTRAINT fk_song FOREIGN KEY (song) REFERENCES songs(song))")
cursor.execute("CREATE TABLE IF NOT EXISTS charts (song text primary key, spotify integer, apple integer, deezer integer, shazam integer, CONSTRAINT fk_song FOREIGN KEY (song) REFERENCES songs(song))")
cursor.execute("CREATE TABLE IF NOT EXISTS concerts (artist_name text, event text primary key, location text, venue text, CONSTRAINT fk_artist FOREIGN KEY (artist_name) REFERENCES artist_songs(artist_name))")

In [14]:
# extracting from csv 1
# csv columns: track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
csv_file = 'spotify-2023.csv'
with open(csv_file, 'r') as csv_data:
    csv_reader = csv.DictReader(csv_data)
    for row in csv_reader:
        # inserting into songs table
        cursor.execute('''
            INSERT OR IGNORE INTO songs (song, day, month, year, streams)
            VALUES (?, ?, ?, ?, ?)
        ''', (row['track_name'], int(row['released_day']), int(row['released_month']), int(row['released_year']), int(row['streams'])))
        # inserting into charts table
        cursor.execute('''
            INSERT OR IGNORE INTO charts (song, spotify, apple, deezer, shazam)
            VALUES (?, ?, ?, ?, ?)
        ''', (row['track_name'], row['in_spotify_charts'], row['in_apple_charts'], row['in_deezer_charts'], row['in_shazam_charts']))


ValueError: invalid literal for int() with base 10: '1,021'

In [10]:
conn.commit()
conn.close()